___
<img style="float: right; margin: 15px 15px 15px 15px;" src="https://cdn.brandfetch.io/idjblR01Uf/w/400/h/400/theme/dark/icon.jpeg?c=1dxbfHSJFAPEGdCLU4o5B" width="200px" height="180px" />

# <font color= #5aafc1> **PROYECTO FINAL** </font>
#### <font color= #2E9AFE> `Red Aeroportuaria Internacional`</font>
- <Strong> Sofía Maldonado, Viviana Toledo, César Adrián Santos </Strong>
- <Strong> Fecha </Strong>: 17/07/2025

___

In [40]:
# Actualizar 
import importlib
import queries 
importlib.reload(queries)
from queries import DB_Queries

# Establecer conexión
db = DB_Queries('neo5j')

Successfully Connected to Neo4j in neo5j database!
Successfully Connected to GraphDataScience in neo5j database!


# <font color= #5aafc1> **1. Exploratory Data Analysis** </font>

Node Count:

In [38]:
db.count_nodes('Airport')

[{'Airports': 3214}]


Relationship Count:

In [ ]:
db.count_relationships('TO')